# Data Querying

You can execute queries and get back the result as a pandas DataFrame.

There are two ways to query data:

- with `cube.query` by passing a list of measure and level objects
- with `session.query_mdx` by passing an MDX string

## Querying current session

In [ ]:
import atoti as tt

session = tt.create_session()
store = session.read_csv("data/sales.csv", keys=["Sale ID"])
cube = session.create_cube(store, "Sales")

### Querying a cube

In [ ]:
m = cube.measures
lvl = cube.levels
cube.query(m["Amount.SUM"], m["Quantity.SUM"], levels=[lvl["Product"]])

### Condition in queries

A condition can be added to filter the query:

In [ ]:
cube.query(
    m["Amount.SUM"],
    m["Quantity.SUM"],
    levels=[lvl["Product"]],
    condition=(lvl["Shop"] == "shop_0"),
)

Use `level.isin` or `hierarchy.isin` to filter on several values:

In [ ]:
cube.query(
    m["Amount.SUM"],
    m["Quantity.SUM"],
    levels=[lvl["Product"]],
    condition=lvl["Product"].isin("TSH_7", "TSH_8", "TSH_9"),
)

Conditions on different hierarchies can be combined with the `&` operator:

In [ ]:
cube.query(
    m["Amount.SUM"],
    m["Quantity.SUM"],
    levels=[lvl["Shop"], lvl["Product"]],
    condition=(
        lvl["Product"].isin("TSH_7", "TSH_8", "TSH_9")
        & lvl["Shop"].isin("shop_0", "shop_1", "shop_2")
    ),
)

### MDX Query

Any MDX query can be used to retrieve data as a pandas DataFrame:

In [ ]:
mdx_query = """
SELECT
  NON EMPTY [Measures].[Amount.SUM] ON COLUMNS,
  NON EMPTY [Sales].[Product].[Product].Members ON ROWS
  FROM (
    SELECT
    {
      [Sales].[Date].[ALL].[AllMember].[2020-05-14],
      [Sales].[Date].[ALL].[AllMember].[2020-05-15],
      [Sales].[Date].[ALL].[AllMember].[2020-05-16]
    } ON COLUMNS
    FROM [Sales]
  )
"""

In [ ]:
session.query_mdx(mdx_query)

## Querying remote cube

It is possible to connect to an existing cube and query it.
This cube can come from another atoti process or even a classic ActivePivot (version >=5.7).

To do that, open a query session, passing the `auth` parameter to indicate how to authenticate against this server.
A query session is immutable: its structure cannot be changed like regular local atoti sessions.

For this example, we will connect to the session created before with atoti:

In [ ]:
existing_session = tt.open_query_session(
    session.url,  # Here you would put the base URL of the existing session.
    # if we needed authentication we could pass it like that
    # auth=tt.query.create_basic_authentication("admin", "admin"),
)
existing_session

### Authentication

atoti provides helpers for 2 authentication schemes:

#### Basic authentication with username and password

In [ ]:
auth = tt.query.create_basic_authentication("admin", "admin")

#### Token based authentication (for instance JWT)

In [ ]:
auth = tt.query.create_token_authentication("some_jwt")

But custom authentication can also be used: `open_query_session`'s `auth` parameter accepts a function taking the server URL and returning a dictionary of HTTP headers to include in the request.

For instance this lambda is similar to `create_token_authentication` and can be used to authenticate with a token:

In [ ]:
auth = lambda url: {"Authorization": f"Bearer {token}"}

### Querying a remote cube

You can retrieve the cubes of the remote session to access their levels and measures and query it.

In [ ]:
existing_cube = existing_session.cubes["Sales"]
lvl = existing_cube.levels
m = existing_cube.measures
existing_cube.query(m["Amount.SUM"], m["Quantity.SUM"], levels=[lvl["Product"]])

MDX queries are also supported:

In [ ]:
existing_session.query_mdx(mdx_query)

And interactive visualizations too:

In [ ]:
existing_session.visualize()